## Optimisation of Staff Scheduling for Restaurants 

Creation of a staff schedule roster for daily operation is a time consuming process for restaurant managers, especially when the restaurant hires a large number of part-time staffs and offers flexible working hours. There are many restrictions that they have to consider, such as staff's availability for the day and the number of staff required at peak and non-peak hours to maintain customer service standards. This often results in ineffective workforce scheduling, which increases the operating expense of the restaurant since salary paid out to staff may not be optimised.

To improve the efficiency of workforce scheduling and better manage operating expenses, an optimisation model can be used by restaurant managers to generate optimal daily working schedules that would effectively utilise available manpower and minimise operating expenses of the restaurant for higher profits. 

Type of model used: Integer Linear Programming
    

#### Packages

In [12]:
import pandas as pd
import numpy as np
import rsome as rso
import numpy as np
from rsome import ro
from rsome import ort_solver as solver

#### Load Datasets
- staff_required: contains the minimum number of service crew required each hour according to historical restaurant foot traffic
- staff_position: contains information on whether service crew is a full-timer (FT) or part-timer (PT) 
- availability: contains availability of all service crew by hour for a particular day 

In [13]:
staff_required = pd.read_csv("Staff Required.csv")
staff_position = pd.read_csv("Staff Position.csv")
availability = pd.read_excel("Restaurant Scheduling Availability.xlsx")

#### Data Preparation

In [14]:
# staff position data
staff_position = staff_position.sort_values(by=['ft_pt','name'], ascending=True)
n_ft = len(staff_position[staff_position['ft_pt']=='FT'])
n_pt = len(staff_position[staff_position['ft_pt']=='PT'])
n_total= len(staff_position)

# availability data 
availability = pd.merge(availability, staff_position, how="left", on = ['name'])
availability_sorted = availability.sort_values(by=['ft_pt','name'], ascending=True, ignore_index=True)


#### Declaring Parameters

In [8]:
# PT/FT salary per hour
c_ft = 16
c_pt = 12

# FT: min working hour requirement
ft_min_hour = 8

# PT: max working hour limit
pt_max_hour = 8

# FT staff requirement per shift
min_ft = 1

# s[i][j]: availability of staff i  at time j
s = np.array(availability_sorted.iloc[:,1:15])

# d[j]: minimum number of staff required at time j
d = np.array(staff_required.set_index('time').T)

# size of the problem 
# n: number of staff
# m: number of working hours/shifts
n,m = np.shape(s) 

#### Optimisation Model
Decision variable $x_{ij}$ is a binary variable that represents whether staff i is assigned shift at time j 

Objective: Minimise total daily salary of full-time and part-time staff 

In [16]:
# Preparing an optimisation model
model = ro.Model('Staff Scheduling')

# Defining decision variables 
# x[i][j]: assign shift to staff i at time j
x = model.dvar((n,m),'B')

# Setting objective to minimise cost
model.min((c_ft*x[0:n_ft]).sum() + (c_pt*x[n_ft:n_total]).sum())


# Adding constraints 

# demand constraint
model.st(sum(x[i,j] for i in range (n)) >= d[0,j] for j in range (m)) 

# availability constraint
model.st(x[i,j] <= s[i,j] for (i,j) in [(i,j) for i in range(n) for j in range(m)])

# at least 1 FT staff constraint
model.st(sum(x[i,j] for i in range (n_ft)) >= min_ft for j in range (m))  

# FT working hour constraint 
model.st(sum(x[i,j] for j in range (m)) >= ft_min_hour for i in range (n_ft)) 

# PT working hour constraint
model.st(sum(x[i,j] for j in range (m)) <= pt_max_hour for i in range (n_ft,n)) 


# Shift restriction contraints: each shift at least 3h
        
#0-1-0 pattern not allowed
for i in range (n):
    for j in range (m-2): 
        model.st(sum(x[i,j:j+3], -x[i,j+1]*2) >= 0)
        
#0-1 pattern at end not allowed    
for i in range (n):
        model.st(sum(x[i,12:13], -x[i,13:14]) >= 0)

# 1-0 pattern at start not allowed
for i in range (n):
        model.st(sum(x[i,0:1], -x[i,1:2]) <=0)
        
#0-1-1 pattern at end not allowed        
for i in range (n):
        model.st(sum(x[i,12:14], -x[i,11:12]) <=1)
        
# 1-1-0 pattern at start not allowed   
for i in range (n):
        model.st(sum(x[i,0:2], -x[i,2:3]) <=1)
        
# 0-1-1-0 pattern not allowed
for i in range (n):
    for j in range (m-4):
        model.st(sum(sum(sum(x[i, j+1:j+2],x[i, j+2:j+3]), -x[i, j+3:j+4]),-x[i, j:j+1]) <=1)
        

#### Optimal Solution
Solving the model to obtain the optimal staff schedule and total salary for the day

In [17]:
# Solving the optimisation model 
model.solve(solver)


#Printing the optimal solution obtained with the input parameters 
print("\nOptimal Solutions:")
for i in range(n):
    print("")
    for j in range(m):
        print(f"Assigned shift at {j+9}00 for", availability_sorted['name'][i] ,f":{format(x.get()[i,j],'1.0f')}")



Being solved by OR-Tools...
Solution status: 0
Running time: 0.1721s

Optimal Solutions:

Assigned shift at 900 for Francis :1
Assigned shift at 1000 for Francis :1
Assigned shift at 1100 for Francis :1
Assigned shift at 1200 for Francis :1
Assigned shift at 1300 for Francis :0
Assigned shift at 1400 for Francis :0
Assigned shift at 1500 for Francis :1
Assigned shift at 1600 for Francis :1
Assigned shift at 1700 for Francis :1
Assigned shift at 1800 for Francis :1
Assigned shift at 1900 for Francis :-0
Assigned shift at 2000 for Francis :-0
Assigned shift at 2100 for Francis :-0
Assigned shift at 2200 for Francis :-0

Assigned shift at 900 for Riley :-0
Assigned shift at 1000 for Riley :0
Assigned shift at 1100 for Riley :0
Assigned shift at 1200 for Riley :1
Assigned shift at 1300 for Riley :1
Assigned shift at 1400 for Riley :1
Assigned shift at 1500 for Riley :1
Assigned shift at 1600 for Riley :1
Assigned shift at 1700 for Riley :1
Assigned shift at 1800 for Riley :0
Assigned shift

####  Daily Staff Schedule
Optimal solution can be put in a dataframe for the restaurant manager to have an overview of schedule for the day

In [76]:
result_df = pd.DataFrame(x.get())
result_df['name'] = pd.DataFrame(availability_sorted['name'])
result_df = result_df.iloc[:, [14,0,1,2,3,4,5,6,7,8,9,10,11,12,13]]
result_df = result_df.set_axis(availability_sorted.columns.values[0:15], axis='columns', inplace=False)
result_df

,name,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00
0,Francis,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
1,Riley,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-0.0,-0.0
2,Sharon,-0.0,-0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
3,Tyson,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Zane,-0.0,-0.0,-0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
5,Alice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0
6,Alyssa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0
7,Andrea,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
8,Britney,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
9,Claudia,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,1.0,1.0,1.0


#### Daily Staff Salary
Total daily salary can also be calculated for the restaurant manager to keep track of operating expenses coming from salaries of employees

In [77]:
# optimal cost
print("\nTotal Daily Salary: $"+format(model.get(),'.2f'))


Total Daily Salary: $1168.00
